In [1]:
# Module Importations
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# Print versioning information
print(sklearn.__version__)

NameError: name 'sklearn' is not defined

In [2]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.features import hyperparameter_opt
from src.models import model_evaluation
from src.models import sklearn_helpers

In [3]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split data into training / evaluation sets
training_set, test_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop profile id data column (both sets)
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

test_set = test_set.drop("profile_id", axis = 1)

ambient        coolant            u_d            u_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.005043       0.005308       0.003792      -0.005359   
std         0.996490       1.002488       0.998747       1.001464   
min        -5.239872      -1.270450      -1.654002      -1.852853   
25%        -0.601386      -1.037872      -0.831843      -0.924537   
50%         0.266671      -0.178200       0.267391      -0.098490   
75%         0.686838       0.673512       0.358587       0.848608   
max         2.954662       2.296845       2.273808       1.788773   

         motor_speed         torque            i_d            i_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.004852      -0.001687       0.004184      -0.001570   
std         1.001954       0.998525       0.999021       0.998389   
min        -1.353747      -3.339106      -3.235659      -3.329749   
25%        -0.951892      -0.267419      -0.75

In [6]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

rotor_test_data = test_set.drop("pm", axis = 1)
rotor_test_label_data = test_set["pm"].copy()

In [7]:
# Linear Regression Model (Target - Rotor Temperature)
lin_reg = LinearRegression()
lin_reg.fit(rotor_training_data, rotor_label_data)

# Name model
model_type = "LinRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(lin_reg, model_name)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print CV results
model_name = "Linear Regression"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae (cv):", scores_mae.std())

# Evaluation
start_time = time.process_time()
rotor_pred_eval = lin_reg.predict(rotor_test_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, rotor_test_label_data, rotor_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_LinRegr_2020_08_18-13_36_53.pkl
Linear Regression rmse mean (cv): 0.4749361664649533
Linear Regression rmse std (cv): 0.0021400886517142672
Linear Regression mae mean (cv): 0.3614280996261001
Linear Regression mae (cv): 0.0016560246676062818
Prediction Time: 0.140625
Linear Regression rmse (Eval): 0.4753533122550699
Linear Regression mae (Eval): 0.36201495208919415
Linear Regression r2 (Eval): 0.7718320737434563


In [8]:
# Polynomial Regression Model (Target - Rotor Temperature)
lin_reg_poly = LinearRegression()

# Create polynomial dataset
poly_features = PolynomialFeatures(degree = 2, include_bias = False)
rotor_training_poly = poly_features.fit_transform(rotor_training_data)

lin_reg_poly.fit(rotor_training_poly, rotor_label_data)

# Name model
model_type = "LinRegrPoly"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(lin_reg_poly, model_name)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg_poly, rotor_training_poly, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg_poly, rotor_training_poly, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Linear Regression - Polynomial"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae std (cv):", scores_mae.std())

# Evaluation
rotor_test_poly = poly_features.fit_transform(rotor_test_data)

start_time = time.process_time()
rotor_pred_eval = lin_reg_poly.predict(rotor_test_poly)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, rotor_test_label_data, rotor_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_LinRegrPoly_2020_08_18-13_36_56.pkl
Linear Regression - Polynomial rmse mean (cv): 0.41047276311126407
Linear Regression - Polynomial rmse std (cv): 0.004807933470279054
Linear Regression - Polynomial mae mean (cv): 0.30105790569514473
Linear Regression - Polynomial mae std (cv): 0.0012163879552887916
Prediction Time: 0.359375
Linear Regression - Polynomial rmse (Eval): 0.4088304713024484
Linear Regression - Polynomial mae (Eval): 0.30157356945924835
Linear Regression - Polynomial r2 (Eval): 0.8312250214320183


In [12]:
# Decision Tree Regressor Model (Target - Rotor Temperature)
tree_regr = DecisionTreeRegressor(max_depth = 10)
tree_regr.fit(rotor_training_data, rotor_label_data)

# Name model
model_type = "TreeRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(tree_regr, model_name)

# Compute RMSE via cross validation
scores_mse = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Decision Tree Regressor"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae std (cv):", scores_mae.std())

# Evaluation
start_time = time.process_time()
rotor_pred_eval = tree_regr.predict(rotor_test_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, rotor_test_label_data, rotor_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_TreeRegr_2020_08_18-13_47_54.pkl
Decision Tree Regressor rmse mean (cv): 0.3504104857362644
Decision Tree Regressor rmse std (cv): 0.005694952390642156
Decision Tree Regressor mae mean (cv): 0.2382755489172744
Decision Tree Regressor mae std (cv): 0.00283718682592299
Prediction Time: 0.109375
Decision Tree Regressor rmse (Eval): 0.3575495777190468
Decision Tree Regressor mae (Eval): 0.2420279645954203
Decision Tree Regressor r2 (Eval): 0.8709095587328388


In [13]:
# Random Forest Regressor Model (Target - Rotor Temperature)
forest_regr = RandomForestRegressor(max_depth = 10)
forest_regr.fit(rotor_training_data, rotor_label_data)

# Name model
model_type = "ForestRegr"
model_name = sklearn_helpers.name_model(model_type)

# Save model
sklearn_helpers.save_model(forest_regr, model_name)

# Compute RMSE via cross validation
scores_mse = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Print results
model_name = "Random Forest Regressor"
print(model_name, "rmse mean (cv):", scores_rmse.mean())
print(model_name, "rmse std (cv):", scores_rmse.std())

print(model_name, "mae mean (cv):", scores_mae.mean())
print(model_name, "mae std (cv):", scores_mae.std())

# Evaluation
start_time = time.process_time()
rotor_pred_eval = forest_regr.predict(rotor_test_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, rotor_test_label_data, rotor_pred_eval)

Save Path: C:\Developer\electric_motor_thermal_modelling\Models\YC_ForestRegr_2020_08_18-13_51_09.pkl
Random Forest Regressor rmse mean (cv): 0.3150629005588953
Random Forest Regressor rmse std (cv): 0.003849557906891769
Random Forest Regressor mae mean (cv): 0.21474772610078707
Random Forest Regressor mae std (cv): 0.001039992296539934
Prediction Time: 7.90625
Random Forest Regressor rmse (Eval): 0.3175761336966866
Random Forest Regressor mae (Eval): 0.21709793443228992
Random Forest Regressor r2 (Eval): 0.8981602649706539


In [14]:
# Random Forest Regressor - Optimised Hyperparameters
forest_regr = RandomForestRegressor(max_depth = 10)

model_name = "Random Forest [Opt]"

best_params, best_est, best_score = hyperparameter_opt.perform_gridsearch(model_name, forest_regr, rotor_training_data, rotor_label_data)

# Evaluation
start_time = time.process_time()
rotor_pred_eval = best_est.predict(rotor_test_data)
end_time = time.process_time()
print("Prediction Time:", (end_time-start_time))

model_evaluation.evaluate_model(model_name, rotor_test_label_data, rotor_pred_eval)

Random Forest [Opt] - Best Params: {'max_features': 6, 'n_estimators': 30}
Random Forest [Opt] - Best Estimator: RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features=6, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
Random Forest [Opt] - Best Score: 0.2932631799720049
Prediction Time: 2.9375
Random Forest [Opt] rmse (Eval): 0.29560805972496706
Random Forest [Opt] mae (Eval): 0.2087737897080213
Random Forest [Opt] r2 (Eval): 0.9117623185065785
